# Eval駆動システム設計：プロトタイプから本番環境まで

## 概要

### このクックブックの目的

このクックブックは、労働集約的な人的ワークフローを置き換える本番グレードの自律システムを作成する際に、evalsを中核プロセスとして効果的に活用する方法について、**実践的**でエンドツーエンドのガイドを提供します。これは、ユーザーが完璧にラベル付けされたデータや問題の完全な理解から始められない場合があるプロジェクトでの共同経験の直接的な成果です。これらの問題は、ほとんどのチュートリアルでは軽視されがちですが、実際にはほぼ常に深刻な課題となります。

evalsを中核プロセスにすることで、当て推量や印象的な精度判断を防ぎ、代わりにエンジニアリングの厳密性を要求します。これにより、コストトレードオフと投資について原則に基づいた決定を下すことができます。

### 対象読者

このガイドは、入門チュートリアルを超えた実践的なガイダンスを求めるML/AIエンジニアやソリューションアーキテクト向けに設計されています。このノートブックは完全に実行可能で、コードサンプルを独自のアプリケーションで直接使用できるよう、可能な限りモジュラーに構成されています。

### ガイドとなる物語：小さな種から本番システムまで

現実的なストーリーラインに従います：経費検証のための手動レシート分析サービスの置き換えです。

* **小さく始める：** 非常に小さなラベル付きデータセット（小売レシート）から始めます。多くの企業は良質なグラウンドトゥルースデータセットを持っていません。
* **段階的に構築：** 最小限の実行可能システムを開発し、初期evalsを確立します。
* **ビジネスとの整合：** ビジネスKPIと金銭的影響の文脈でeval性能を評価し、低影響の改善作業を避けるよう努力を集中させます。
* **Eval駆動の反復：** evalスコアを使用してモデル改善を推進し、その後より良いモデルをより多くのデータで使用してevalsを拡張し、さらなる改善領域を特定することで、反復的に改善します。

### このクックブックの使用方法

このクックブックは、LLMアプリケーション構築のライフサイクルを通じたeval中心のガイドとして構成されています。

1. 主に提示されたアイデアに興味がある場合は、テキストを読み、コードを軽く確認してください。
2. 他の作業中のプロジェクトのためにここにいる場合は、該当するセクションに直接ジャンプしてそこのコードを詳しく調べ、コピーして自分のニーズに適応させることができます。
3. これがどのように動作するかを本当に理解したい場合は、このノートブックをダウンロードして読みながらセルを実行し、コードを編集して独自の変更を加え、仮説をテストし、すべてがどのように連携するかを実際に理解していることを確認してください。

> 注意：OpenAI組織がゼロデータ保持（ZDR）ポリシーを持っている場合、Evalsは引き続き利用可能ですが、アプリケーション状態を維持するためにデータを保持します。

## ユースケース：レシート解析

このガイドを簡潔にするため、詳細で多面的な評価に値する小さな仮想的な問題を使用します。特に、限られたデータ量で問題を解決する方法に焦点を当てるため、非常に小さなデータセットを使用します。

### 問題定義

このガイドでは、レシートの確認と整理のワークフローから始めることを想定しています。一般的に、これは既に多くの確立されたソリューションが存在する問題ですが、これまでにそれほど多くの先行研究がない他の問題と類似しています。さらに、優れたエンタープライズソリューションが存在する場合でも、人間の時間を必要とする「ラストマイル」の問題が残ることがよくあります。

私たちのケースでは、以下のようなパイプラインがあると仮定します：

* 人々がレシートの写真をアップロードする
* 経理チームが各レシートを確認して分類し、経費を承認または監査する

経理チームへのインタビューに基づくと、彼らは以下の要素に基づいて判断を行います：

1. 商店
2. 地理的位置
3. 経費金額
4. 購入した商品またはサービス
5. 手書きのメモや注釈

私たちのシステムは、人間の介入なしにほとんどのレシートを処理することが期待されますが、信頼度の低い判断については人間のQAにエスカレーションします。私たちは経理プロセスの総コストを削減することに焦点を当てており、これは以下に依存します：

1. 以前/現在のシステムがレシート1枚あたりにかかる運用コスト
2. 新しいシステムがQAに送るレシートの数
3. システムのレシート1枚あたりの運用コストと固定費
4. ミスのビジネスへの影響（確認のために除外されたレシートまたは見逃されたミス）
5. システムの開発と統合にかかるエンジニアリングコスト

### データセット概要

レシート画像は、RoboflowによってCC by 4.0ライセンスで公開された[Receipt Handwriting Detection Computer Vision Project](https://universe.roboflow.com/newreceipts/receipt-handwriting-detection)データセットから取得しています。少数の例でストーリーを伝えるために、独自のラベルと物語的な解釈を追加しました。

## プロジェクトライフサイクル

すべてのプロジェクトが同じ方法で進行するわけではありませんが、プロジェクトには一般的にいくつかの重要な共通要素があります。

![Project Lifecycle](../../../images/partner_project_lifecycle.png)

実線の矢印は主要な進行やステップを示し、点線は問題理解の継続的な性質を表しています。つまり、顧客ドメインについてより多くを発見することが、プロセスのすべてのステップに影響を与えるということです。以下では、これらの反復的な改善サイクルのいくつかを詳しく検討します。
すべてのプロジェクトが同じ方法で進行するわけではありませんが、プロジェクトには一般的にいくつかの重要な共通要素があります。

### 1. 問題を理解する

通常、エンジニアリングプロセスを開始する決定は、ビジネスインパクトを理解しているがプロセスの詳細を知る必要のないリーダーシップによって行われます。我々の例では、非AIワークフローを置き換えるように設計されたシステムを構築しています。ある意味でこれは理想的です：ドメインエキスパートのセット、つまり*現在そのタスクを実行している人々*がいて、彼らにインタビューしてタスクの詳細を理解し、適切なevalsの開発を支援してもらうことができます。

このステップは、システムの構築を開始する前に終了するものではありません。必然的に、我々の初期評価は問題空間の不完全な理解であり、解決策に近づくにつれて理解を継続的に改善していくことになります。

### 2. 例を収集する（データを集める）

実世界のプロジェクトが、満足のいく解決策を達成するために必要なすべてのデータ、ましてや信頼性を確立するためのデータを持って開始されることは非常に稀です。

我々のケースでは、レシート画像の形でシステム*入力*の適切なサンプルがあると仮定しますが、完全に注釈付けされたデータなしで開始します。これは既存のプロセスを自動化する際の珍しくない状況だと我々は考えています。進行に伴ってドメインエキスパートと協力してテストセットとトレーニングセットを段階的に拡張し、evalsを徐々により包括的にするプロセスを説明します。

### 3. エンドツーエンドV0システムを構築する

可能な限り迅速にシステムの骨格を構築したいと考えています。優れたパフォーマンスを発揮するシステムは必要ありません。適切な入力を受け入れ、正しいタイプの出力を提供するものがあれば十分です。通常、これはプロンプトでタスクを記述し、入力を追加し、単一のモデル（通常は構造化出力を使用）を使って初期のベストエフォート試行を行うのとほぼ同じくらい簡単です。

### 4. データにラベルを付け、初期Evalsを構築する

確立された正解データがない場合、システムの初期バージョンを使用して「ドラフト」の正解データを生成し、それをドメインエキスパートが注釈付けまたは修正することは珍しくないことがわかっています。

エンドツーエンドシステムが構築されたら、持っている入力を処理して妥当な出力を生成し始めることができます。これらをドメインエキスパートに送って評価と修正をしてもらいます。これらの修正と、エキスパートがどのように決定を下しているかについての会話を使用して、さらなるevalsを設計し、システムに専門知識を組み込みます。

### 5. Evalsをビジネスメトリクスにマッピングする

すべてのエラーの修正に飛び込む前に、時間を効果的に投資していることを確認する必要があります。この段階での最も重要なタスクは、evalsをレビューし、それらが主要な目標にどのように関連しているかを理解することです。

- 一歩下がってシステムの潜在的なコストと利益を評価する
- どのeval測定がそれらのコストと利益に直接関連するかを特定する
- 例えば、特定のevalでの「失敗」にはどのようなコストがかかるか？価値のあることを測定しているか？
- evalメトリクスを使用してドル価値を提供する（非LLM）モデルを作成する
- パフォーマンス（精度、または速度）と開発・運用コストのバランスを取る

### 6. システムとEvalsを段階的に改善する

どの努力が最も価値があるかを特定したら、システムの改善に関する反復を開始できます。evalsは客観的なガイドとして機能し、システムが十分に良くなったときを知ることができ、回帰を回避または特定することを確実にします。

### 7. QAプロセスと継続的改善を統合する

Evalsは開発だけのものではありません。本番サービスの全部または一部を計装することで、時間の経過とともにより有用なテストとトレーニングサンプルが浮上し、不正確な仮定を特定したり、カバレッジが不十分な領域を見つけたりできます。これは、初期開発プロセスが完了してからずっと後でも、モデルが良好なパフォーマンスを継続していることを確実にする唯一の方法でもあります。

## V0システム構築

実際には、おそらくREST APIを介して動作するシステムを構築し、一連のコンポーネントやリソースにアクセスできるWebフロントエンドを備えることになるでしょう。このクックブックの目的では、それを`extract_receipt_details`と`evaluate_receipt_for_audit`という2つの関数に集約し、これらが連携して特定のレシートに対して何をすべきかを決定します。

- `extract_receipt_details`は画像を入力として受け取り、レシートに関する重要な詳細を含む構造化された出力を生成します。
- `evaluate_receipt_for_audit`はその構造を入力として受け取り、レシートを監査すべきかどうかを決定します。

> このようにプロセスを段階に分けることには長所と短所の両方があります。小さく独立した段階で構成されていれば、プロセスの検証と開発が容易になります。しかし、段階的に情報を失う可能性があり、実質的にエージェントに「伝言ゲーム」をさせることになります。このノートブックでは段階を分けて、監査者に実際のレシートを見せないようにしています。これは、議論したい評価についてより教育的だからです。

最初のステップである文字通りのデータ抽出から始めます。これは*中間*データです。人々が暗黙的に検証する情報ですが、しばしば記録されません。そのため、作業の基となるラベル付きデータがないことが多いのです。

In [ ]:
%pip install --upgrade openai pydantic python-dotenv rich persist-cache -qqq
%load_ext dotenv
%dotenv

# Place your API key in a file called .env
# OPENAI_API_KEY=sk-...

### 構造化出力モデル

構造化された出力で意味のある情報を取得します。

In [ ]:
from pydantic import BaseModel


class Location(BaseModel):
    city: str | None
    state: str | None
    zipcode: str | None


class LineItem(BaseModel):
    description: str | None
    product_code: str | None
    category: str | None
    item_price: str | None
    sale_price: str | None
    quantity: str | None
    total: str | None


class ReceiptDetails(BaseModel):
    merchant: str | None
    location: Location
    time: str | None
    items: list[LineItem]
    subtotal: str | None
    tax: str | None
    total: str | None
    handwritten_notes: list[str]

> *注意*: 通常、上記の数値には`decimal.Decimal`オブジェクトを使用し、`time`フィールドには`datetime.datetime`オブジェクトを使用しますが、これらはどちらもデシリアライゼーションがうまくいきません。このクックブックの目的上、文字列を使用しますが、実際には正しい出力を検証するために別のレベルの変換を行う必要があります。

### 基本情報の抽出

`extract_receipt_details`関数を構築しましょう。

通常、何かが機能する可能性のある最初の試みでは、これまでに収集した利用可能なドキュメントをChatGPTに単純に入力し、プロンプトを生成するよう依頼します。ベンチマークとして自分を評価する基準を持つ前に、プロンプトエンジニアリングに多くの時間を費やす価値はありません！これは、上記の問題説明に基づいてo4-miniによって生成されたプロンプトです。

In [ ]:
BASIC_PROMPT = """
Given an image of a retail receipt, extract all relevant information and format it as a structured response.

# Task Description

Carefully examine the receipt image and identify the following key information:

1. Merchant name and any relevant store identification
2. Location information (city, state, ZIP code)
3. Date and time of purchase
4. All purchased items with their:
   * Item description/name
   * Item code/SKU (if present)
   * Category (infer from context if not explicit)
   * Regular price per item (if available)
   * Sale price per item (if discounted)
   * Quantity purchased
   * Total price for the line item
5. Financial summary:
   * Subtotal before tax
   * Tax amount
   * Final total
6. Any handwritten notes or annotations on the receipt (list each separately)

## Important Guidelines

* If information is unclear or missing, return null for that field
* Format dates as ISO format (YYYY-MM-DDTHH:MM:SS)
* Format all monetary values as decimal numbers
* Distinguish between printed text and handwritten notes
* Be precise with amounts and totals
* For ambiguous items, use your best judgment based on context

Your response should be structured and complete, capturing all available information
from the receipt.
"""

In [ ]:
import base64
import mimetypes
from pathlib import Path

from openai import AsyncOpenAI

client = AsyncOpenAI()


async def extract_receipt_details(
    image_path: str, model: str = "o4-mini"
) -> ReceiptDetails:
    """Extract structured details from a receipt image."""
    # Determine image type for data URI.
    mime_type, _ = mimetypes.guess_type(image_path)

    # Read and base64 encode the image.
    b64_image = base64.b64encode(Path(image_path).read_bytes()).decode("utf-8")
    image_data_url = f"data:{mime_type};base64,{b64_image}"

    response = await client.responses.parse(
        model=model,
        input=[
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": BASIC_PROMPT},
                    {"type": "input_image", "image_url": image_data_url},
                ],
            }
        ],
        text_format=ReceiptDetails,
    )

    return response.output_parsed

### 1つのレシートでテスト

1つのレシートだけを評価し、手動でレビューして、単純なプロンプトを使ったスマートモデルがどの程度うまく動作するかを確認してみましょう。

<img src="../../../images/Supplies_20240322_220858_Raven_Scan_3_jpeg.rf.50852940734939c8838819d7795e1756.jpg" alt="Walmart_image" width="400"/>

（注：この画像タグは翻訳対象外のため、そのまま保持されています）

In [ ]:
from rich import print

receipt_image_dir = Path("data/test")
ground_truth_dir = Path("data/ground_truth")

example_receipt = Path(
    "data/train/Supplies_20240322_220858_Raven_Scan_3_jpeg.rf.50852940734939c8838819d7795e1756.jpg"
)
result = await extract_receipt_details(example_receipt)

再実行すると異なる回答が得られますが、通常はいくつかのエラーがあるものの、ほとんどのことは正しく処理されます。以下は具体的な例です：

In [ ]:
walmart_receipt = ReceiptDetails(
    merchant="Walmart",
    location=Location(city="Vista", state="CA", zipcode="92083"),
    time="2023-06-30T16:40:45",
    items=[
        LineItem(
            description="SPRAY 90",
            product_code="001920056201",
            category=None,
            item_price=None,
            sale_price=None,
            quantity="2",
            total="28.28",
        ),
        LineItem(
            description="LINT ROLLER 70",
            product_code="007098200355",
            category=None,
            item_price=None,
            sale_price=None,
            quantity="1",
            total="6.67",
        ),
        LineItem(
            description="SCRUBBER",
            product_code="003444193232",
            category=None,
            item_price=None,
            sale_price=None,
            quantity="2",
            total="12.70",
        ),
        LineItem(
            description="FLOUR SACK 10",
            product_code="003444194263",
            category=None,
            item_price=None,
            sale_price=None,
            quantity="1",
            total="0.77",
        ),
    ],
    subtotal="50.77",
    tax="4.19",
    total="54.96",
    handwritten_notes=[],
)

モデルは多くの項目を正しく抽出しましたが、一部の行項目を誤って名前変更しました。さらに重要なことに、一部の価格を間違って取得し、どの行項目もカテゴリ分けしないことを決定しました。

それは問題ありません。この時点で完璧な答えを期待しているわけではありません！代わりに、私たちの目標は評価可能な基本システムを構築することです。そして、反復を開始するときには、*見た目が*良いものに向かって「感覚的に」進むのではなく、信頼性の高いソリューションを工学的に構築することになります。しかし、まず、ドラフトシステムを完成させるためにアクション決定を追加します。

### アクション決定

次に、ループを閉じて、レシートに基づいた実際の決定を行う必要があります。これは非常に似ているため、コメントなしでコードを提示します。

通常は、最も高性能なモデル（現時点では`o3`）から始めて最初のパスを行い、正確性が確立されたら、異なるモデルで実験してビジネスへの影響のトレードオフを分析し、反復によって改善可能かどうかを検討します。クライアントは、低レイテンシやコストのために一定の精度の低下を受け入れる場合もあれば、コスト、レイテンシ、精度の目標を達成するためにアーキテクチャを変更する方が効果的な場合もあります。これらのトレードオフを明示的かつ客観的に行う方法については、後で詳しく説明します。

このクックブックでは、`o3`は優秀すぎるかもしれません。最初のパスでは`o4-mini`を使用して、いくつかの推論エラーを発生させ、それらが発生した際の対処方法を説明するために使用します。

次に、ループを閉じて、レシートに基づいた実際の決定を行う必要があります。これは非常に似ているため、コメントなしでコードを提示します。

In [ ]:
from pydantic import BaseModel, Field

audit_prompt = """
Evaluate this receipt data to determine if it need to be audited based on the following
criteria:

1. NOT_TRAVEL_RELATED:
   - IMPORTANT: For this criterion, travel-related expenses include but are not limited
   to: gas, hotel, airfare, or car rental.
   - If the receipt IS for a travel-related expense, set this to FALSE.
   - If the receipt is NOT for a travel-related expense (like office supplies), set this
   to TRUE.
   - In other words, if the receipt shows FUEL/GAS, this would be FALSE because gas IS
   travel-related.

2. AMOUNT_OVER_LIMIT: The total amount exceeds $50

3. MATH_ERROR: The math for computing the total doesn't add up (line items don't sum to
   total)

4. HANDWRITTEN_X: There is an "X" in the handwritten notes

For each criterion, determine if it is violated (true) or not (false). Provide your
reasoning for each decision, and make a final determination on whether the receipt needs
auditing. A receipt needs auditing if ANY of the criteria are violated.

Return a structured response with your evaluation.
"""


class AuditDecision(BaseModel):
    not_travel_related: bool = Field(
        description="True if the receipt is not travel-related"
    )
    amount_over_limit: bool = Field(description="True if the total amount exceeds $50")
    math_error: bool = Field(description="True if there are math errors in the receipt")
    handwritten_x: bool = Field(
        description="True if there is an 'X' in the handwritten notes"
    )
    reasoning: str = Field(description="Explanation for the audit decision")
    needs_audit: bool = Field(
        description="Final determination if receipt needs auditing"
    )


async def evaluate_receipt_for_audit(
    receipt_details: ReceiptDetails, model: str = "o4-mini"
) -> AuditDecision:
    """Determine if a receipt needs to be audited based on defined criteria."""
    # Convert receipt details to JSON for the prompt
    receipt_json = receipt_details.model_dump_json(indent=2)

    response = await client.responses.parse(
        model=model,
        input=[
            {
                "role": "user",
                "content": [
                    {"type": "input_text", "text": audit_prompt},
                    {"type": "input_text", "text": f"Receipt details:\n{receipt_json}"},
                ],
            }
        ],
        text_format=AuditDecision,
    )

    return response.output_parsed

全体的なプロセスの概略図では、2つのLLM呼び出しが示されています：

![Process Flowchart](../../../images/partner_process_flowchart.png)

上記の例をこのモデルで実行すると、以下のような結果が得られます。ここでも例の結果を使用します。コードを実行する際には、わずかに異なる結果が得られる可能性があります。

In [ ]:
audit_decision = await evaluate_receipt_for_audit(result)
print(audit_decision)

In [ ]:
audit_decision = AuditDecision(
    not_travel_related=True,
    amount_over_limit=True,
    math_error=False,
    handwritten_x=False,
    reasoning="""
    The receipt from Walmart is for office supplies, which are not travel-related, thus NOT_TRAVEL_RELATED is TRUE.
    The total amount of the receipt is $54.96, which exceeds the limit of $50, making AMOUNT_OVER_LIMIT TRUE.
    The subtotal ($50.77) plus tax ($4.19) correctly sums to the total ($54.96), so there is no MATH_ERROR.
    There are no handwritten notes, so HANDWRITTEN_X is FALSE.
    Since two criteria (amount over limit and travel-related) are violated, the receipt
    needs auditing.
    """,
    needs_audit=True,
)

この例は、なぜエンドツーエンドの評価を重視し、なぜそれらを単独で使用できないのかを示しています。ここでは、初期の抽出にOCRエラーがあり、合計に合わない価格を監査者に転送しましたが、監査者はそれを検出できず、数学的エラーはないと断言しています。しかし、これを見逃しても監査決定は変わりません。なぜなら、レシートが監査される必要がある他の2つの理由を捉えていたからです。

したがって、`AuditDecision`は事実上正しくありませんが、私たちが重視する決定は正しいのです。これは改善すべき点を示してくれますが、同時にエンジニアリングの努力をどこで、いつ適用するかについて適切な選択をするよう導いてくれます。

それでは、評価システムを構築してみましょう！

## 初期評価

最小限の機能を持つシステムができたら、より多くの入力を処理し、ドメインエキスパートに協力してもらって正解データの開発を進めるべきです。専門的なタスクを行うドメインエキスパートは、私たちのプロジェクトに多くの時間を割けない可能性があるため、効率的に進めて小さく始め、最初は深さよりも幅を重視することを目指します。

> あなたのデータがドメインの専門知識を*必要としない*場合は、ラベリングソリューション（[Label Studio](https://labelstud.io/)など）を利用し、ポリシー、予算、データ利用可能性の制約の範囲内で可能な限り多くのデータにアノテーションを付けることを試みるでしょう。
> このケースでは、データラベリングが希少なリソースであると仮定して進めます。つまり、毎週少量であれば頼ることができるが、これらの人々には他の業務責任があり、時間と協力への意欲が限られている可能性があるリソースです。これらのエキスパートと一緒に座ってサンプルのアノテーションを手伝ってもらうことで、将来のサンプル選択をより効率的にできます。

2つのステップからなるチェーンがあるため、`[FilePath, ReceiptDetails, AuditDecision]`型のタプルを収集することになります。一般的に、これを行う方法は、ラベルなしのサンプルを取得し、それらをモデルに通し、その後エキスパートに出力を修正してもらうことです。このノートブックの目的では、`data/test`内のすべてのレシート画像に対してすでにそのプロセスを実行済みです。

### 追加の考慮事項

しかし、マルチステッププロセスを評価する際には、エンドツーエンドのパフォーマンスと、*前のステップの出力を条件とした*各個別ステップのパフォーマンスの両方を知ることが重要であるため、それ以上のことがあります。

このケースでは、以下を評価したいと考えています：

1. 入力画像が与えられたとき、必要な情報をどの程度うまく抽出できるか？
2. レシート情報が与えられたとき、監査決定に対する私たちの**判断**はどの程度良いか？
3. 入力画像が与えられたとき、最終的な監査決定を行うことにどの程度**成功**しているか？

#2と#3の表現の違いは、監査者に間違ったデータを与えた場合、間違った結論に達することが予想されるためです。私たちが*望む*のは、たとえその証拠が誤解を招くものであっても、監査者が利用可能な証拠に基づいて正しい決定を下していることを確信することです。このケースに注意を払わないと、監査者が入力を無視するように訓練してしまい、全体的なパフォーマンスが低下する原因となる可能性があります。

### 採点者

evalの中核となるコンポーネントは[採点者](https://platform.openai.com/docs/guides/graders)です。最終的なevalでは18個の採点者を使用しますが、使用するのは3種類のみで、いずれも概念的に非常にわかりやすいものです。

以下に、文字列チェック採点者の例、テキスト類似度採点者の例、そして最後にモデル採点者の例を示します。

In [ ]:
example_graders = [
    {
        "name": "Total Amount Accuracy",
        "type": "string_check",
        "operation": "eq",
        "input": "{{ item.predicted_receipt_details.total }}",
        "reference": "{{ item.correct_receipt_details.total }}",
    },
    {
        "name": "Merchant Name Accuracy",
        "type": "text_similarity",
        "input": "{{ item.predicted_receipt_details.merchant }}",
        "reference": "{{ item.correct_receipt_details.merchant }}",
        "pass_threshold": 0.8,
        "evaluation_metric": "bleu",
    },
]

# A model grader needs a prompt to instruct it in what it should be scoring.
missed_items_grader_prompt = """
Your task is to evaluate the correctness of a receipt extraction model.

The following items are the actual (correct) line items from a specific receipt.

{{ item.correct_receipt_details.items }}

The following items are the line items extracted by the model.

{{ item.predicted_receipt_details.items }}

Score 0 if the sample evaluation missed any items from the receipt; otherwise score 1.

The line items are permitted to have small differences or extraction mistakes, but each
item from the actual receipt must be present in some form in the model's output. Only
evaluate whether there are MISSED items; ignore other mistakes or extra items.
"""

example_graders.append(
    {
        "name": "Missed Line Items",
        "type": "score_model",
        "model": "o4-mini",
        "input": [{"role": "system", "content": missed_items_grader_prompt}],
        "range": [0, 1],
        "pass_threshold": 1,
    }
)

各グレーダーは予測された出力の一部を評価します。これは構造化された出力の特定のフィールドに対する非常に狭い範囲のチェックかもしれませんし、出力全体を判断するより包括的なチェックかもしれません。一部のグレーダーはコンテキストなしで動作し、出力を単独で評価できます（例えば、段落が失礼または不適切かどうかを評価するLLM判定器）。他のものは入力と出力に基づいて評価できますが、ここで使用しているものは出力と比較対象となる正解（正しい）出力に依存しています。

Evalsを使用する最も直接的な方法は、プロンプトとモデルを提供し、evalが入力に対して実行されて出力を自分で生成できるようにすることです。もう一つの有用な方法は、以前にログされた応答や補完を出力のソースとして使用することです。それほど単純ではありませんが、最も柔軟にできることは、使用したいすべてを含むアイテムを提供することです。これにより、「予測」機能を単一のモデル呼び出しに制限するのではなく、任意のシステムにすることができます。これが以下の例での使用方法です。下記に示す`EvaluationRecord`は、`{{ }}`テンプレート変数を埋めるために使用されます。

> **モデル選択に関する注意：**  
> 適切なモデルを選択することは重要です。より高速で安価なモデルは本番環境では好ましいことが多いですが、開発ワークフローでは利用可能な最も高性能なモデルを優先することが有益です。このガイドでは、システムタスクとLLMベースのグレーディングの両方に`o4-mini`を使用しています。`o3`はより高性能ですが、私たちの経験では、出力品質の差は大幅なコスト増加に比べて控えめです。実際には、エンジニア1人あたり1日10ドル以上をevalsに費やすのは一般的ですが、エンジニア1人あたり1日100ドル以上に拡大することは持続可能ではないかもしれません。
>
> それでも、`o3`のようなより高度なモデルで定期的にベンチマークを取ることは価値があります。大幅な改善が観察された場合は、評価データの代表的なサブセットに組み込むことを検討してください。モデル間の不一致は重要なエッジケースを明らかにし、システム改善の指針となります。

In [ ]:
import asyncio


class EvaluationRecord(BaseModel):
    """Holds both the correct (ground truth) and predicted audit decisions."""

    receipt_image_path: str
    correct_receipt_details: ReceiptDetails
    predicted_receipt_details: ReceiptDetails
    correct_audit_decision: AuditDecision
    predicted_audit_decision: AuditDecision


async def create_evaluation_record(image_path: Path, model: str) -> EvaluationRecord:
    """Create a ground truth record for a receipt image."""
    extraction_path = ground_truth_dir / "extraction" / f"{image_path.stem}.json"
    correct_details = ReceiptDetails.model_validate_json(extraction_path.read_text())
    predicted_details = await extract_receipt_details(image_path, model)

    audit_path = ground_truth_dir / "audit_results" / f"{image_path.stem}.json"
    correct_audit = AuditDecision.model_validate_json(audit_path.read_text())
    predicted_audit = await evaluate_receipt_for_audit(predicted_details, model)

    return EvaluationRecord(
        receipt_image_path=image_path.name,
        correct_receipt_details=correct_details,
        predicted_receipt_details=predicted_details,
        correct_audit_decision=correct_audit,
        predicted_audit_decision=predicted_audit,
    )


async def create_dataset_content(
    receipt_image_dir: Path, model: str = "o4-mini"
) -> list[dict]:
    # Assemble paired samples of ground truth data and predicted results. You could
    # instead upload this data as a file and pass a file id when you run the eval.
    tasks = [
        create_evaluation_record(image_path, model)
        for image_path in receipt_image_dir.glob("*.jpg")
    ]
    return [{"item": record.model_dump()} for record in await asyncio.gather(*tasks)]


file_content = await create_dataset_content(receipt_image_dir)

グレーダーとデータが準備できたら、評価の作成と実行は非常に簡単です：

In [ ]:
from persist_cache import cache


# We're caching the output so that if we re-run this cell we don't create a new eval.
@cache
async def create_eval(name: str, graders: list[dict]):
    eval_cfg = await client.evals.create(
        name=name,
        data_source_config={
            "type": "custom",
            "item_schema": EvaluationRecord.model_json_schema(),
            "include_sample_schema": False,  # Don't generate new completions.
        },
        testing_criteria=graders,
    )
    print(f"Created new eval: {eval_cfg.id}")
    return eval_cfg


initial_eval = await create_eval(
    "Initial Receipt Processing Evaluation", example_graders
)

# Run the eval.
eval_run = await client.evals.runs.create(
    name="initial-receipt-processing-run",
    eval_id=initial_eval.id,
    data_source={
        "type": "jsonl",
        "source": {"type": "file_content", "content": file_content},
    },
)
print(f"Evaluation run created: {eval_run.id}")
print(f"View results at: {eval_run.report_url}")

その評価を実行した後、UIで確認することができ、以下のような画面が表示されるはずです。

（注意：Zero-Data-Retention契約を結んでいる場合、このデータはOpenAIによって保存されないため、このインターフェースでは利用できません。）

![Summary UI](../../../images/partner_summary_ui.png)

データタブをクリックして、個別の例を詳しく確認することができます：

![Details UI](../../../images/partner_details_ui.png)

## 評価をビジネス指標に結び付ける

評価は改善できる箇所を示し、時間の経過とともに進歩と後退を追跡するのに役立ちます。
しかし、上記の3つの評価は単なる測定値です。これらに存在意義を与える必要があります。

まず必要なのは、レシート処理の最終段階に対する評価を追加することです。これにより、監査決定の結果を確認し始めることができます。次に必要なもの、そして最も重要なのは、*ビジネス関連性のモデル*です。

### ビジネスモデル

新しいシステムがどの程度うまく機能するかに応じて、どのようなコストと利益が得られるかを算出するのは、ほぼ決して簡単ではありません。多くの場合、人々は不確実性がどれほど大きいかを知っており、自分を悪く見せるような推測をしたくないため、物事に数値を付けることを避けようとします。それは問題ありません。最善の推測をするだけで、後でより多くの情報が得られれば、モデルを改良することができます。

このクックブックでは、シンプルなコスト構造を作成します：

- 当社は年間100万件のレシートを処理し、ベースラインコストはレシート1件あたり0.20ドル
- レシートの監査には約2ドルのコストがかかる
- 監査すべきレシートの監査に失敗すると、平均30ドルのコストがかかる
- レシートの5%が監査を必要とする
- 既存のプロセスは
  - 監査が必要なレシートを97%の確率で特定する
  - 監査が不要なレシートを2%の確率で誤って特定する

これにより、2つのベースライン比較が得られます：

- すべてのレシートを正しく特定できれば、監査に10万ドルを費やすことになる
- 現在のプロセスは監査に13万5千ドルを費やし、監査されていない経費で4万5千ドルを失っている

さらに、人間主導のプロセスには追加で20万ドルのコストがかかります。

当社のサービスは、運用コストが安い（上記のプロンプトを`o4-mini`で使用した場合、レシート1件あたり約1セント）ことで費用を節約することを期待していますが、監査と見逃された監査でお金を節約するか損失するかは、システムがどの程度うまく機能するかによって決まります。これを簡単な関数として書く価値があるかもしれません。以下に書かれているのは、上記の要因を含むが、ニュアンスを無視し、開発、保守、サービス提供のコストを無視したバージョンです。

In [ ]:
def calculate_costs(fp_rate: float, fn_rate: float, per_receipt_cost: float):
    audit_cost = 2
    missed_audit_cost = 30
    receipt_count = 1e6
    audit_fraction = 0.05

    needs_audit_count = receipt_count * audit_fraction
    no_needs_audit_count = receipt_count - needs_audit_count

    missed_audits = needs_audit_count * fn_rate
    total_audits = needs_audit_count * (1 - fn_rate) + no_needs_audit_count * fp_rate

    audit_cost = total_audits * audit_cost
    missed_audit_cost = missed_audits * missed_audit_cost
    processing_cost = receipt_count * per_receipt_cost

    return audit_cost + missed_audit_cost + processing_cost


perfect_system_cost = calculate_costs(0, 0, 0)
current_system_cost = calculate_costs(0.02, 0.03, 0.20)

print(f"Current system cost: ${current_system_cost:,.0f}")

### Evalsへの接続

上記のモデルのポイントは、単なる数値に過ぎないevalに意味を与えることができることです。例えば、上記のシステムを実行した際、商店名について85%の時間で間違っていました。しかし詳しく調べてみると、ほとんどのケースは大文字小文字の問題や「Shell Gasoline」対「Shell Oil #2144」といった問題で、これらを追跡してみると監査決定に影響を与えたり、基本的なコストを変更したりすることはないようです。

一方で、レシート上の手書きの「X」を捉え損なうことが約半分の時間で発生し、レシート上の「X」が見逃された場合の約半分で、本来監査されるべきレシートが監査されないという結果になるようです。これらはデータセット内で過大に表現されていますが、これがレシートの1%を占めるだけでも、その50%の失敗は年間75,000ドルのコストとなります。

同様に、計算が合わないことを理由に、OCRエラーによってレシートを監査することが最大20%の時間で頻繁に発生しているようです。これは約400,000ドルのコストになる可能性があります！

今、私たちはより多くのgraderを追加し、監査決定の精度から逆算して、どの問題に焦点を当てるべきかを決定する立場にあります。

以下は残りのgraderと、最初の最適化されていないプロンプトで得られた結果です。この時点では非常に悪い結果であることに注意してください！20サンプル（8つの陽性、12つの陰性）全体で、2つの偽陰性と2つの偽陽性がありました。これを事業全体に外挿すると、見逃した監査で375,000ドル、不要な監査で475,000ドルを失うことになります。

In [ ]:
simple_extraction_graders = [
    {
        "name": "Merchant Name Accuracy",
        "type": "text_similarity",
        "input": "{{ item.predicted_receipt_details.merchant }}",
        "reference": "{{ item.correct_receipt_details.merchant }}",
        "pass_threshold": 0.8,
        "evaluation_metric": "bleu",
    },
    {
        "name": "Location City Accuracy",
        "type": "string_check",
        "operation": "eq",
        "input": "{{ item.predicted_receipt_details.location.city }}",
        "reference": "{{ item.correct_receipt_details.location.city }}",
    },
    {
        "name": "Location State Accuracy",
        "type": "string_check",
        "operation": "eq",
        "input": "{{ item.predicted_receipt_details.location.state }}",
        "reference": "{{ item.correct_receipt_details.location.state }}",
    },
    {
        "name": "Location Zipcode Accuracy",
        "type": "string_check",
        "operation": "eq",
        "input": "{{ item.predicted_receipt_details.location.zipcode }}",
        "reference": "{{ item.correct_receipt_details.location.zipcode }}",
    },
    {
        "name": "Time Accuracy",
        "type": "string_check",
        "operation": "eq",
        "input": "{{ item.predicted_receipt_details.time }}",
        "reference": "{{ item.correct_receipt_details.time }}",
    },
    {
        "name": "Subtotal Amount Accuracy",
        "type": "string_check",
        "operation": "eq",
        "input": "{{ item.predicted_receipt_details.subtotal }}",
        "reference": "{{ item.correct_receipt_details.subtotal }}",
    },
    {
        "name": "Tax Amount Accuracy",
        "type": "string_check",
        "operation": "eq",
        "input": "{{ item.predicted_receipt_details.tax }}",
        "reference": "{{ item.correct_receipt_details.tax }}",
    },
    {
        "name": "Total Amount Accuracy",
        "type": "string_check",
        "operation": "eq",
        "input": "{{ item.predicted_receipt_details.total }}",
        "reference": "{{ item.correct_receipt_details.total }}",
    },
    {
        "name": "Handwritten Notes Accuracy",
        "type": "text_similarity",
        "input": "{{ item.predicted_receipt_details.handwritten_notes }}",
        "reference": "{{ item.correct_receipt_details.handwritten_notes }}",
        "pass_threshold": 0.8,
        "evaluation_metric": "fuzzy_match",
    },
]

item_extraction_base = """
Your task is to evaluate the correctness of a receipt extraction model.

The following items are the actual (correct) line items from a specific receipt.

{{ item.correct_receipt_details.items }}

The following items are the line items extracted by the model.

{{ item.predicted_receipt_details.items }}
"""

missed_items_instructions = """
Score 0 if the sample evaluation missed any items from the receipt; otherwise score 1.

The line items are permitted to have small differences or extraction mistakes, but each
item from the actual receipt must be present in some form in the model's output. Only
evaluate whether there are MISSED items; ignore other mistakes or extra items.
"""

extra_items_instructions = """
Score 0 if the sample evaluation extracted any extra items from the receipt; otherwise
score 1.

The line items are permitted to have small differences or extraction mistakes, but each
item from the actual receipt must be present in some form in the model's output. Only
evaluate whether there are EXTRA items; ignore other mistakes or missed items.
"""

item_mistakes_instructions = """
Score 0 to 10 based on the number and severity of mistakes in the line items.

A score of 10 means that the two lists are perfectly identical.

Remove 1 point for each minor mistake (typos, capitalization, category name
differences), and up to 3 points for significant mistakes (incorrect quantity, price, or
total, or categories that are not at all similar).
"""

item_extraction_graders = [
    {
        "name": "Missed Line Items",
        "type": "score_model",
        "model": "o4-mini",
        "input": [
            {
                "role": "system",
                "content": item_extraction_base + missed_items_instructions,
            }
        ],
        "range": [0, 1],
        "pass_threshold": 1,
    },
    {
        "name": "Extra Line Items",
        "type": "score_model",
        "model": "o4-mini",
        "input": [
            {
                "role": "system",
                "content": item_extraction_base + extra_items_instructions,
            }
        ],
        "range": [0, 1],
        "pass_threshold": 1,
    },
    {
        "name": "Item Mistakes",
        "type": "score_model",
        "model": "o4-mini",
        "input": [
            {
                "role": "system",
                "content": item_extraction_base + item_mistakes_instructions,
            }
        ],
        "range": [0, 10],
        "pass_threshold": 8,
    },
]


simple_audit_graders = [
    {
        "name": "Not Travel Related Accuracy",
        "type": "string_check",
        "operation": "eq",
        "input": "{{ item.predicted_audit_decision.not_travel_related }}",
        "reference": "{{ item.correct_audit_decision.not_travel_related }}",
    },
    {
        "name": "Amount Over Limit Accuracy",
        "type": "string_check",
        "operation": "eq",
        "input": "{{ item.predicted_audit_decision.amount_over_limit }}",
        "reference": "{{ item.correct_audit_decision.amount_over_limit }}",
    },
    {
        "name": "Math Error Accuracy",
        "type": "string_check",
        "operation": "eq",
        "input": "{{ item.predicted_audit_decision.math_error }}",
        "reference": "{{ item.correct_audit_decision.math_error }}",
    },
    {
        "name": "Handwritten X Accuracy",
        "type": "string_check",
        "operation": "eq",
        "input": "{{ item.predicted_audit_decision.handwritten_x }}",
        "reference": "{{ item.correct_audit_decision.handwritten_x }}",
    },
    {
        "name": "Needs Audit Accuracy",
        "type": "string_check",
        "operation": "eq",
        "input": "{{ item.predicted_audit_decision.needs_audit }}",
        "reference": "{{ item.correct_audit_decision.needs_audit }}",
    },
]


reasoning_eval_prompt = """
Your task is to evaluate the quality of *reasoning* for audit decisions on receipts.
Here are the rules for audit decisions:

Expenses should be audited if they violate any of the following criteria:
1. Expenses must be travel-related
2. Expenses must not exceed $50
3. All math should be correct; the line items plus tax should equal the total
4. There must not be an "X" in the handwritten notes

If ANY of those criteria are violated, the expense should be audited.

Here is the input to the grader:
{{ item.predicted_receipt_details }}

Below is the output of an authoritative grader making a decision about whether or not to
audit an expense. This is a correct reference decision.

GROUND TRUTH:
{{ item.correct_audit_decision }}


Here is the output of the model we are evaluating:

MODEL GENERATED:
{{ item.predicted_audit_decision }}


Evaluate:
1. For each of the 4 criteria, did the model correctly score it as TRUE or FALSE?
2. Based on the model's *scoring* of the criteria (regardless if it scored it
   correctly), did the model reason appropriately about the criteria (i.e. did it
   understand and apply the prompt correctly)?
3. Is the model's reasoning logically sound, sufficient, and comprehensible?
4. Is the model's reasoning concise, without extraneous details?
5. Is the final decision to audit or not audit correct?

Grade the model with the following rubric:
- (1) point for each of the 4 criteria that the model scored correctly
- (3) points for each aspect of the model's reasoning that is meets the criteria
- (3) points for the model's final decision to audit or not audit

The total score is the sum of the points, and should be between 0 and 10 inclusive.
"""


model_judgement_graders = [
    {
        "name": "Audit Reasoning Quality",
        "type": "score_model",
        "model": "o4-mini",
        "input": [{"role": "system", "content": reasoning_eval_prompt}],
        "range": [0, 10],
        "pass_threshold": 8,
    },
]

full_eval = await create_eval(
    "Full Receipt Processing Evaluation",
    simple_extraction_graders
    + item_extraction_graders
    + simple_audit_graders
    + model_judgement_graders,
)

eval_run = await client.evals.runs.create(
    name="complete-receipt-processing-run",
    eval_id=full_eval.id,
    data_source={
        "type": "jsonl",
        "source": {"type": "file_content", "content": file_content},
    },
)

eval_run.report_url

![Large Summary UI](../../../images/partner_large_summary_ui.png)

## フライホイールを回す

ビジネスモデルを持つことで、何をする価値があり、何をする価値がないかのマップを手に入れることができます。初期の評価（evals）は、私たちが正しい方向に進んでいることを知らせる道路標識です。しかし、最終的にはより多くの標識が必要になります。プロセスのこの段階では、通常、取り組むべき多くの異なることがあり、いくつかの連結したサイクルがあります。そこでは、一つの改善が別のサイクルでのさらなる改善の余地を開くことになります。

![Development Flywheel](../../../images/partner_development_flywheel.png)

1. 評価は改善できる箇所を示してくれ、モデル選択、プロンプトエンジニアリング、ツール使用、ファインチューニング戦略の指針として即座に活用できます。
2. システムが評価に従って良好に動作するようになっても、それで終わりではありません。その時こそ*評価を改善する*時です。より多くのデータを処理し、ドメインエキスパートにレビューしてもらい、修正をより良く包括的な評価の構築にフィードバックします。

このサイクルはしばらく続けることができます。検査する「興味深い」データの効率的フロンティアを特定することで、このサイクルを加速できます。これにはいくつかの技術がありますが、簡単な方法の一つは、一貫した答えが得られない入力のラベリングを優先するために、入力に対してモデルを再実行することです。これは異なる基盤モデルを使用する際に特に効果的で、しばしば知能の低いモデルを使用することからも恩恵を受けます（愚かなモデルが賢いモデルと一致するなら、それはおそらく難しい問題ではありません）。

パフォーマンスの収穫逓減点に達したと思われる時点で、同じ技術を使ってモデルコストを最適化し続けることができます。十分に良好に動作するシステムがあれば、ファインチューニングや何らかの形のモデル蒸留により、より小さく、安価で、高速なモデルから同様のパフォーマンスを得ることができるでしょう。

## システム改善

評価が整い、それらがビジネス指標とどのように関連しているかを理解できたので、ついにシステムの出力を改善することに注意を向ける準備が整いました。

上記で述べたように、加盟店名を85%の確率で間違えており、これは評価している他のどの出力よりも多くなっています。これはかなり悪く見え、おそらく少しの作業で劇的に改善できるものですが、代わりにビジネス指標のエンドポイントから始めて、不正確な決定を引き起こした問題を見つけるために逆算してみましょう。

そうすると、加盟店名で犯した間違いは最終的な監査決定と完全に無相関であり、それらがその決定に何らかの影響を与えているという証拠はないことがわかります。私たちのビジネスモデルに基づくと、実際にはそれを改善する必要性は見当たりません。つまり、*すべての評価が重要というわけではない*のです。代わりに、悪い監査決定を下した具体的な例を検証することができます。それらは（20件中）わずか2件しかありません。それらを詳しく調べると、どちらの場合も問題のない抽出に基づいてパイプラインの第2段階が間違った決定を下したことが問題だったことがわかります。そして実際、両方とも旅行関連費用について正しく推論できなかったことに起因しています。

最初のケースでは、購入品は自動車部品店のスノーブラシです。これは少しエッジケースですが、私たちのドメインエキスパートはこれを有効な旅行費用として特定しました（ドライバーがフロントガラスを清掃するために必要になる可能性があるため）。決定プロセスをより詳細に説明し、類似の例を提供することで、このエラーを修正できるようです。

2番目のケースでは、購入品はホームセンターの工具です。これらの工具は通常の運転とは何の関係もないため、このレシートは「旅行関連でない費用」として監査されるべきです。この場合、私たちのモデルは旅行関連でない費用として*正しく*特定しますが、その事実について間違って推論し、`not_travel_related`の`true`が`needs_audit`の`true`を意味するべきだということを明らかに誤解しています。これも、指示をより明確にし、いくつかの例を示すことで問題を解決できそうな例です。

これをコストモデルに関連付けると、偽陰性が1件、偽陽性が1件、真陽性が7件、真陰性が11件あることがわかります。これを本番環境で見られる頻度に外挿すると、年間の全体的なコストが63,000ドル増加することになります。

プロンプトを修正し、評価を再実行して結果を確認してみましょう。エンジンオイルに関する具体的な例を指示に含めてより多くのガイダンスを提供し（スノーブラシとは異なりますが、同じ推論が必要です）、トレーニングセット（`data/train`）から抽出した3つの例をfew-shotガイダンスとして含めます。

In [ ]:
first_ai_system_cost = calculate_costs(
    fp_rate=1 / 12, fn_rate=1 / 8, per_receipt_cost=0.01
)

print(f"First version of our system, estimated cost: ${first_ai_system_cost:,.0f}")

In [ ]:
nursery_receipt_details = ReceiptDetails(
    merchant="WESTERN SIERRA NURSERY",
    location=Location(city="Oakhurst", state="CA", zipcode="93644"),
    time="2024-09-27T12:33:38",
    items=[
        LineItem(
            description="Plantskydd Repellent RTU 1 Liter",
            product_code=None,
            category="Garden/Pest Control",
            item_price="24.99",
            sale_price=None,
            quantity="1",
            total="24.99",
        )
    ],
    subtotal="24.99",
    tax="1.94",
    total="26.93",
    handwritten_notes=[],
)

nursery_audit_decision = AuditDecision(
    not_travel_related=True,
    amount_over_limit=False,
    math_error=False,
    handwritten_x=False,
    reasoning="""
    1. The merchant is a plant nursery and the item purchased an insecticide, so this
       purchase is not travel-related (criterion 1 violated).
    2. The total is $26.93, under $50, so criterion 2 is not violated.
    3. The line items (1 * $24.99 + $1.94 tax) sum to $26.93, so criterion 3 is not
       violated.
    4. There are no handwritten notes or 'X's, so criterion 4 is not violated.
    Since NOT_TRAVEL_RELATED is true, the receipt must be audited.
    """,
    needs_audit=True,
)

flying_j_details = ReceiptDetails(
    merchant="Flying J #616",
    location=Location(city="Frazier Park", state="CA", zipcode=None),
    time="2024-10-01T13:23:00",
    items=[
        LineItem(
            description="Unleaded",
            product_code=None,
            category="Fuel",
            item_price="4.459",
            sale_price=None,
            quantity="11.076",
            total="49.39",
        )
    ],
    subtotal="49.39",
    tax=None,
    total="49.39",
    handwritten_notes=["yos -> home sequoia", "236660"],
)
flying_j_audit_decision = AuditDecision(
    not_travel_related=False,
    amount_over_limit=False,
    math_error=False,
    handwritten_x=False,
    reasoning="""
    1. The only item purchased is Unleaded gasoline, which is travel-related so
       NOT_TRAVEL_RELATED is false.
    2. The total is $49.39, which is under $50, so AMOUNT_OVER_LIMIT is false.
    3. The line items ($4.459 * 11.076 = $49.387884) sum to the total of $49.39, so
       MATH_ERROR is false.
    4. There is no "X" in the handwritten notes, so HANDWRITTEN_X is false.
    Since none of the criteria are violated, the receipt does not need auditing.
    """,
    needs_audit=False,
)

engine_oil_details = ReceiptDetails(
    merchant="O'Reilly Auto Parts",
    location=Location(city="Sylmar", state="CA", zipcode="91342"),
    time="2024-04-26T8:43:11",
    items=[
        LineItem(
            description="VAL 5W-20",
            product_code=None,
            category="Auto",
            item_price="12.28",
            sale_price=None,
            quantity="1",
            total="12.28",
        )
    ],
    subtotal="12.28",
    tax="1.07",
    total="13.35",
    handwritten_notes=["vista -> yos"],
)
engine_oil_audit_decision = AuditDecision(
    not_travel_related=False,
    amount_over_limit=False,
    math_error=False,
    handwritten_x=False,
    reasoning="""
    1. The only item purchased is engine oil, which might be required for a vehicle
       while traveling, so NOT_TRAVEL_RELATED is false.
    2. The total is $13.35, which is under $50, so AMOUNT_OVER_LIMIT is false.
    3. The line items ($12.28 + $1.07 tax) sum to the total of $13.35, so
       MATH_ERROR is false.
    4. There is no "X" in the handwritten notes, so HANDWRITTEN_X is false.
    None of the criteria are violated so the receipt does not need to be audited.
    """,
    needs_audit=False,
)

examples = [
    {"input": nursery_receipt_details, "output": nursery_audit_decision},
    {"input": flying_j_details, "output": flying_j_audit_decision},
    {"input": engine_oil_details, "output": engine_oil_audit_decision},
]

# Format the examples as JSON, with each example wrapped in XML tags.
example_format = """
<example>
    <input>
        {input}
    </input>
    <output>
        {output}
    </output>
</example>
"""

examples_string = ""
for example in examples:
    example_input = example["input"].model_dump_json()
    correct_output = example["output"].model_dump_json()
    examples_string += example_format.format(input=example_input, output=correct_output)

audit_prompt = f"""
Evaluate this receipt data to determine if it need to be audited based on the following
criteria:

1. NOT_TRAVEL_RELATED:
   - IMPORTANT: For this criterion, travel-related expenses include but are not limited
   to: gas, hotel, airfare, or car rental.
   - If the receipt IS for a travel-related expense, set this to FALSE.
   - If the receipt is NOT for a travel-related expense (like office supplies), set this
   to TRUE.
   - In other words, if the receipt shows FUEL/GAS, this would be FALSE because gas IS
   travel-related.
   - Travel-related expenses include anything that could be reasonably required for
   business-related travel activities. For instance, an employee using a personal
   vehicle might need to change their oil; if the receipt is for an oil change or the
   purchase of oil from an auto parts store, this would be acceptable and counts as a
   travel-related expense.

2. AMOUNT_OVER_LIMIT: The total amount exceeds $50

3. MATH_ERROR: The math for computing the total doesn't add up (line items don't sum to
   total)
   - Add up the price and quantity of each line item to get the subtotal
   - Add tax to the subtotal to get the total
   - If the total doesn't match the amount on the receipt, this is a math error
   - If the total is off by no more than $0.01, this is NOT a math error

4. HANDWRITTEN_X: There is an "X" in the handwritten notes

For each criterion, determine if it is violated (true) or not (false). Provide your
reasoning for each decision, and make a final determination on whether the receipt needs
auditing. A receipt needs auditing if ANY of the criteria are violated.

Note that violation of a criterion means that it is `true`. If any of the above four
values are `true`, then the receipt needs auditing (`needs_audit` should be `true`: it
functions as a boolean OR over all four criteria).

If the receipt contains non-travel expenses, then NOT_TRAVEL_RELATED should be `true`
and therefore NEEDS_AUDIT must also be set to `true`. IF THE RECEIPT LISTS ITEMS THAT
ARE NOT TRAVEL-RELATED, THEN IT MUST BE AUDITED. Here are some example inputs to
demonstrate how you should act:

<examples>
{examples_string}
</examples>

Return a structured response with your evaluation.
"""

上記のプロンプトに対して行った修正は以下の通りです：

1. 旅行関連費用に関する項目1の下に、箇条書きを追加しました

```
- Travel-related expenses include anything that could be reasonably required for
  business-related travel activities. For instance, an employee using a personal
  vehicle might need to change their oil; if the receipt is for an oil change or the
  purchase of oil from an auto parts store, this would be acceptable and counts as a
  travel-related expense.
```

2. 数学的エラーを評価する方法について、より規範的なガイダンスを追加しました。
   具体的には、以下の箇条書きを追加しました：

```
   - Add up the price and quantity of each line item to get the subtotal
   - Add tax to the subtotal to get the total
   - If the total doesn't match the amount on the receipt, this is a math error
   - If the total is off by no more than $0.01, this is NOT a math error
```

   これは実際には前述した問題とは関係ありませんが、監査モデルが提供する推論の欠陥として気づいた別の問題です。

3. 旅行関連以外の費用は監査すべきであることを示す非常に強いガイダンス（実際に強調して述べ、再度強調する必要がありました）を追加しました。

```
Note that violation of a criterion means that it is `true`. If any of the above four
values are `true`, then the receipt needs auditing (`needs_audit` should be `true`: it
functions as a boolean OR over all four criteria).

If the receipt contains non-travel expenses, then NOT_TRAVEL_RELATED should be `true`
and therefore NEEDS_AUDIT must also be set to `true`. IF THE RECEIPT LISTS ITEMS THAT
ARE NOT TRAVEL-RELATED, THEN IT MUST BE AUDITED.
```

4. XMLタグで囲まれた3つの例、JSON入力/出力ペアを追加しました。
3. XMLタグで囲まれた3つの例、JSON入力/出力ペアを追加しました。

プロンプトの修正により、評価用のデータを再生成し、同じ評価を再実行して結果を比較します：

In [ ]:
file_content = await create_dataset_content(receipt_image_dir)

eval_run = await client.evals.runs.create(
    name="updated-receipt-processing-run",
    eval_id=full_eval.id,
    data_source={
        "type": "jsonl",
        "source": {"type": "file_content", "content": file_content},
    },
)

eval_run.report_url

評価を再度実行したところ、実際にはまだ2つの監査判定で間違いがありました。間違いを犯した例を詳しく調べてみると、特定した問題は完全に修正されていましたが、例の改善により推論ステップが向上し、他の2つの問題が表面化したことが判明しました。具体的には：

1. 1つのレシートは、抽出時のミスにより手書きの「X」が識別されなかったため、監査が必要でした。監査モデルは正しく推論しましたが、不正確なデータに基づいていました。
2. 1つのレシートは、$0.35のデビット手数料が見えないように抽出されたため、監査モデルが計算エラーを特定しました。これは、より詳細な指示と、計算エラーがあるかどうかを判断するために実際にすべての行項目を合計する必要があることを示す明確な例を提供したために起こったと考えられます。これも監査モデルの正しい動作を示しており、抽出モデルを修正する必要があることを示唆しています。

これは素晴らしいことで、問題を発見し次第、継続的に改善を重ねていきます。これが改善のサイクルです！

### モデルの選択

プロジェクトを開始する際、通常は`o4-mini`などの最も高性能なモデルの1つから始めて、パフォーマンスのベースラインを確立します。モデルがタスクを解決する能力に確信が持てたら、次のステップは、より小さく、高速で、またはコスト効率の良い代替案を探ることです。

推論コストとレイテンシの最適化は、特に本番環境や顧客向けシステムにおいて不可欠です。これらの要因は全体的な費用とユーザーエクスペリエンスに大きな影響を与える可能性があります。例えば、`o4-mini`から`gpt-4.1-mini`に切り替えることで、推論コストを約3分の2削減できる可能性があります。これは、慎重なモデル選択が意味のある節約につながる例です。

次のセクションでは、抽出と監査の両方のステップで`gpt-4.1-mini`を使用して評価を再実行し、より効率的なモデルがどの程度のパフォーマンスを発揮するかを確認します。

In [ ]:
file_content = await create_dataset_content(receipt_image_dir, model="gpt-4.1-mini")

eval_run = await client.evals.runs.create(
    name="receipt-processing-run-gpt-4-1-mini",
    eval_id=full_eval.id,
    data_source={
        "type": "jsonl",
        "source": {"type": "file_content", "content": file_content},
    },
)

eval_run.report_url

結果は非常に有望です。抽出精度は全く低下していないようです。1つの回帰（再びsnowbroom）が見られますが、監査の判定は以前のプロンプト変更前と比べて2倍の頻度で正確になっています。

![Eval Variations](../../../images/partner_eval_variations.png)

これは、より安価なモデルに切り替えることができるという素晴らしい証拠ですが、より多くのプロンプトエンジニアリング、ファインチューニング、または何らかの形のモデル蒸留が必要になる可能性があります。ただし、現在のモデルによると、これによって既にコストを削減できることに注意してください。サンプルサイズが十分に大きくないため、まだ完全には信じていません。実際の偽陰性率は、ここで見られる0よりも高くなるでしょう。

In [ ]:
system_cost_4_1_mini = calculate_costs(
    fp_rate=1 / 12, fn_rate=0, per_receipt_cost=0.003
)

print(f"Cost using gpt-4.1-mini: ${system_cost_4_1_mini:,.0f}")

### さらなる改善

このクックブックは評価の哲学と実践に焦点を当てており、モデル改善技術の全範囲を扱うものではありません。モデルのパフォーマンスを向上または維持するため（特により小さく、高速で、安価なモデルに移行する場合）、以下の手順を順番に検討してください。上から始めて、必要な場合のみ下に進んでください。例えば、ファインチューニングに頼る前に必ずプロンプトを最適化してください。弱いプロンプトでファインチューニングを行うと、後でプロンプトを改善しても悪いパフォーマンスが固定化される可能性があります。

![Model Improvement Waterfall](../../../images/partner_model_improvement_waterfall.png)

1. **モデル選択：** より賢いモデルを試すか、推論予算を増やす。
2. **プロンプトチューニング：** 指示を明確にし、非常に明示的なルールを提供する。
3. **例とコンテキスト：** few-shotまたはmany-shotの例を追加するか、問題により多くのコンテキストを提供する。RAGはここに該当し、類似の例を動的に選択するために使用される場合がある。
4. **ツール使用：** 外部APIへのアクセス、データベースのクエリ機能、またはモデルが自身の質問に答えられるようにする機能を含む、特定の問題を解決するためのツールを提供する。
5. **補助モデル：** 限定的なサブタスクを実行するモデル、監督とガードレールを提供するモデルを追加するか、エキスパートの混合を使用して複数のサブモデルからの解決策を集約する。
6. **ファインチューニング：** 教師ありファインチューニング用のラベル付きトレーニングデータ、強化ファインチューニング用の評価グレーダー、または直接選好最適化用の異なる出力を使用する。

上記のオプションはすべてパフォーマンスを最大化するためのツールです。価格対パフォーマンス比を最適化しようとする場合、通常は上記のすべてを既に実行しており、ほとんどの手順を繰り返す必要はありませんが、より小さなモデルをファインチューニングしたり、最良のモデルを使用してより小さなモデルを訓練する（モデル蒸留）ことは可能です。

> OpenAI Evalsの本当に優れた点の一つは、同じグレーダーを[強化ファインチューニング](https://cookbook.openai.com/examples/reinforcement_fine_tuning)に使用して、極めてサンプル効率的な方法でより良いモデルパフォーマンスを生み出せることです。注意点として、別々のトレーニングデータを使用し、RFT中に評価データセットをリークさせないようにしてください。

## デプロイと開発後の運用
LLMアプリケーションの構築とデプロイは始まりに過ぎません。真の価値は継続的な改善から生まれます。システムが稼働したら、継続的な監視を優先しましょう：トレースをログに記録し、出力を追跡し、スマートサンプリング技術を使用して実際のユーザーインタラクションを積極的にサンプリングして人間によるレビューを行います。

本番データは、評価および訓練データセットを進化させるための最も信頼できるソースです。実際のユースケースから新鮮なサンプルを定期的に収集・整理して、ギャップ、エッジケース、改善の新たな機会を特定しましょう。

実際には、このデータを活用して迅速な反復を行います。最近の高品質なサンプルでモデルを再訓練し、評価で既存のモデルを上回った場合に新しいバージョンを自動的にデプロイする定期的なファインチューニングパイプラインを自動化します。ユーザーの修正とフィードバックを収集し、これらの洞察を体系的にプロンプトや再訓練プロセスにフィードバックします—特に持続的な問題を浮き彫りにする場合は重要です。

これらのフィードバックループを開発後のワークフローに組み込むことで、LLMアプリケーションが継続的に適応し、堅牢性を保ち、進化するユーザーニーズと密接に連携し続けることを確実にします。

### 貢献者
このクックブックは、OpenAIと[Fractional](https://www.fractional.ai/)の共同協力による成果です。

- Hugh Wimberly
- Joshua Marker
- Eddie Siegel
- Shikhar Kwatra